## Project idea 4:
### implement and test at least 6 different variants of Isomap for DGP: the three above, and at least three new ones of your own conception

In [ ]:

import sys
import numpy as np
import math
import types
import matplotlib.pyplot as plt
import networkx as nx
import random
from math import e
from re import M, T


### Isomap for DGP


In [ ]:
def biconnected_graph(n,bool=False):
    "" "Generate a random biconnected graph with n nodes" ""
    X = nx.gnp_random_graph(n, 0.5, directed=False)
    while(not nx.is_biconnected(X)):
        X = nx.gnp_random_graph(n, 0.5, directed=False)
    for (u, v) in X.edges():
        X[u][v]['weight'] = random.randint(1, 10)
    if bool:
        nx.draw(X, with_labels=True)
        plt.show()
    return X

def create_ampl_dat_grah(D, filename):
    "" "Create a .dat file for the ampl model" ""
    with open(filename, 'w') as file:
        file.write("# gen by python\n")
        file.write("param Kdim := 3;\n")
        file.write("param n := " + str(len(D.nodes())) + ";\n")
        file.write("param : E : c I :=\n")
        for (u, v) in D.edges():
            file.write(""+str(u+1) + " " + str(v+1) + " " + str(D[u][v]['weight'])+ " "+"1\n")
        file.write(";\n")
        file.close()
        
def floyd_warshall(D):
    D_prime= nx.to_numpy_array(D)
    "" "Floyd-Warshall algorithm that take an nx and return an array" ""
    for i in range(D_prime.shape[0]):
        for j in range(D_prime.shape[1]):
            if i != j and D_prime[i][j] == 0:
                D_prime[i][j] = math.inf
    for i in range(D_prime.shape[0]):
        for j in range(D_prime.shape[1]):
                for k in range(D_prime.shape[0]):
                    if i != k and j != k and i !=j:
                        if D_prime[i][j] > D_prime[i][k] + D_prime[k][j]:
                            D_prime[i][j] = D_prime[i][k] + D_prime[k][j]
    if np.array_equal(D_prime, D_prime.T):
        return D_prime
    else:
        print("error:The graph is not symetric")
        return None
    
## return the distance matrix of a realization
def distanceMatrix(x, p=2):
    n = len(x[:,0])
    D = np.zeros((n,n))
    for u in range(n-1):
        for v in range(u+1,n):
            D[u,v] = np.linalg.norm(np.subtract(x[u,:],x[v,:]), ord=p)
            D[v,u] = D[u,v]
    return D

## convert a distance matrix to a Gram matrix
def dist2Gram(D):
    n = D.shape[0]
    J = np.identity(n) - (1.0/n)*np.ones((n,n))
    G = -0.5 * np.dot(J,np.dot(np.square(D), J))
    return G

## factor a square matrix
def factor(A):
    n = A.shape[0]
    (evals,evecs) = np.linalg.eigh(A)
    evals[evals < 0] = 0  # closest SDP matrix
    X = evecs #np.transpose(evecs)
    sqrootdiag = np.eye(n)
    for i in range(n):
        sqrootdiag[i,i] = math.sqrt(evals[i])
    X = X.dot(sqrootdiag)
    return np.fliplr(X)

## classic Multidimensional scaling
def MDS(B, eps = 1e-9):
    n = B.shape[0]
    x = factor(B)
    (evals,evecs) = np.linalg.eigh(x)
    K = len(evals[evals > eps])
    if K < n:
        # only first K columns
        x = x[:,0:K]
    return x

## principal component analysis
def PCA(B, K):
    x = factor(B)
    n = B.shape[0]
    if isinstance(K, str):
        K = n
    if K < n:
        # only first K columns
        x = x[:,0:K]
    return x

def complete_PCA_part(D,eps,representation=True):
    G=dist2Gram(D)
    X=MDS(G,eps)
    n=X.shape[0]
    K=X.shape[1]
    print("dimension can be reduced from", n, "to", K)

    if representation:
        if K > 3:
            K = 3
        elif K < 2:
            K = 2
        print("representing in", K, "dimensions")

        X = PCA(G,K)

        if K == 2:
            plt.scatter(X[:,0], X[:,1])
        elif K == 3:
            fig = plt.figure()
            ax = plt.axes(projection='3d')
            ax.scatter(X[:,0], X[:,1], X[:,2])
            ax.plot(X[:,0], X[:,1], X[:,2])
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            ax.set_zlabel('z')

        plt.show()
    else:
        X=PCA(G,K)
    return X



In [ ]:
D=biconnected_graph(10)
create_ampl_dat_grah(D, "graph.dat")

In [ ]:

# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

P = ampl_notebook(
    modules=["baron", "gurobi"],  # modules to install (guroby is the best solver)
    license_uuid="bf5b5ed2-ea35-4373-8776-1ab021939a37",  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6415.6806 expiring 20240515: INF580 Large-scale optimization, Leo Liberti, LIX Ecole Polytechnique.


In [ ]:
%%ampl_eval
reset;

# DGP formulation 4 (pull-and-push)
				 
## the protein graph format
param Kdim integer, > 0;
param n integer, > 0;
set V := 1..n;
set E within {V,V};
param c{E};
param I{E};

# set of dimensions
set K := 1..Kdim;

## decision variables
# positions of vertices (k-th component of position of vertex v)
var x{V,K};

maximize pull: sum{(u,v) in E} sum{k in K} (x[u,k] - x[v,k])^2;

subject to push{(u,v) in E}: sum{k in K} (x[u,k] - x[v,k])^2 <= c[u,v]^2;

#subject to centroid{k in K}: sum{v in V} x[v,k] = 0;


In [ ]:
P.readData("./graph.dat")


BARON 23.3.11 (2023.03.11):  
*** A potentially catastrophic access violation just took place 
*** The search will be terminated by BARON after returning the best known solution 
*** Please report this problem to Nick Sahinidis (niksah@minlp.com) 
 
error running baronin:
	termination code 11
0 iterations, interrupted (Control-C).
Objective 0


In [ ]:
P.setOption("solver", "gurobi")
P.solve()